In [1]:
#Dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import datetime as datetime
import time
import json
import re


In [3]:
#create splinter browser
executable_path = {'executable_path':"../resources/chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [155]:
#define url as variable and open with browser
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [156]:
#grab articles
articles= soup.find_all(class_ = "slide")

In [157]:
#grab headline
headline = articles[0].find(class_= "content_title").text.replace("\n", "")
headline

"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [158]:
#grab abstract
abstract = articles[0].find(class_ = "rollover_description_inner").text.replace("\n", "")
abstract

'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'

In [159]:
#grab article url
articleURL = f"https://mars.nasa.gov{articles[0].find(class_ = 'content_title').a['href']}"
articleURL

'https://mars.nasa.gov/news/8699/how-nasas-mars-helicopter-will-reach-the-red-planets-surface/'

In [160]:
#Loop to store all headlines abstracts and urls

#headlines = []
#abstracts = []
#article_urls = []
#for article in articles:
#    headlines.append(article.find(class_= "content_title").text.replace("\n", ""))
#    abstracts.append(article.find(class_ = "rollover_description_inner").text.replace("\n", ""))
#    article_urls.append("https://mars.nasa.gov" + article.find(class_ = 'content_title').a['href'])


In [162]:
#scrap image url
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [164]:
#find image url
images = soup.find_all(class_ = 'carousel_item')
img_url = "https://www.jpl.nasa.gov" + images[0]['style'].split("'")[1]
img_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23354-1920x1200.jpg'

In [81]:
#scrap mars weather from tweeterz
tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [85]:
#collect all spans to find tweets
spans = soup.find_all('span')

In [109]:
#loop to find first tweet in span
tweeterz = ""
for tweet in spans:
    if 'InSight sol' in tweet.text:
        tweeterz = tweet.text
        #break loop after condition is met
        break
tweeterz

'InSight sol 564 (2020-06-28) low -89.3ºC (-128.8ºF) high -4.3ºC (24.2ºF)\nwinds from the SSW at 5.0 m/s (11.3 mph) gusting to 15.4 m/s (34.4 mph)\npressure at 7.70 hPa'

In [112]:
#find tweet url
oaklandAs = soup.find_all('a')
tweetbook = ''
for link in oaklandAs:
    if 'status' in link['href']:
        tweetbook = 'https://www.twitter.com' + link['href']
        break
tweetbook

'https://www.twitter.com/MarsWxReport/status/1277283412705972226'

In [114]:
#scrape facts
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [115]:
#read all tables from html into dfs
df_list = pd.read_html(html)


In [123]:
df_list[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [125]:
#rename columns
df_list[0].columns = ['Fact', 'Value']
df_list[0]

,Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [127]:
#convert to html and json
fact_html = df_list[0].to_html(index=False)
fact_json = json.loads(df_list[0].to_json(orient='records'))

In [131]:
#scrap hemisphere img and text
hemiURL = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemiURL)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [136]:
#loop to find urls for each hemispehere
hemipages = soup.find_all(class_ = 'item')
hemi_urls = []
for page in hemipages:
    hemi_urls.append("https://astrogeology.usgs.gov/" + page.find('a')['href'])



In [145]:
#scrap title and img url from each page
hemi_titles = []
hemi_imgs = []
for link in hemi_urls:
    browser.visit(link)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    hemi_titles.append(soup.title.text.split(" |")[0])
    hemi_imgs.append(soup.find(class_ = 'downloads').find('a')['href'])